# Creating the Database
---


Citi Bike provides a valuable tranportation service to people who live in or travel to NYC. It has an [open Citi Bike database](https://ride.citibikenyc.com/system-data) and it's free for the public to [download](https://s3.amazonaws.com/tripdata/index.html). In this notebook, we will build a SQL database populated with the data. Since the data format was changed in Feburary 2021, we'll focus on the NYC's trip data before 2021.
  
In this notebook, we will build up a webscraper to pull all the necessary data. After downloading the data, we will merge files and import the data into a PostgreSQL database. 
  
⚠️A new data format of Citi Bike trip data has been used from February 2021.  
⚠️If you encounter issues when you run the codes. Please go and check [Issues](https://github.com/Fitzmon/CitiBike_Analysis/issues?q=is%3Aissue+is%3Aclosed) page.
  


---
Refenrence: 
- [Building a Citibike Database with Python](https://medium.com/@fausto.manon/building-a-citibike-database-with-python-9849a59fb90c)

---
## Table of Contents

- [01. Importing Libraries](#01.-Importing-Libraries)
- [02. Data Preparation](#02.-Data-Preparation)
- [03. Building a PostgreSQL Database](#03.-Building-a-PostgreSQL-Database)
- [04. Unsettled]

---
### 01. Importing Libraries

In [1]:
# Web scraping libraries
import requests
import urllib.request
from bs4 import BeautifulSoup

# Downloading, moving and unzipping files
import webbrowser
from time import sleep
import shutil 
import os
from zipfile import ZipFile

# DataFrame exploration and manipulation
import pandas as pd
from glob import glob

# PostgreSQL interaction
import psycopg2
import linecache
from psycopg2 import sql
from psycopg2 import Error
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

---
### 02. Data Preparation

In [2]:
url = 'https://s3.amazonaws.com/tripdata/'
response = requests.get(url)

In [3]:
soup = BeautifulSoup(response.text, 'xml')

In [4]:
data_files = soup.find_all('Key')

In the step below, we're going to retreive all of the zip files from the site. As Citi Bike also extends to Jersey City and Hoboken in New Jersey, we will filter the data.

In [6]:
# Instantiate empty list
zip_files = []
filter_files = []

# Populate list with zip file names
for file in range(len(data_files)-1):
    zip_files.append(data_files[file].get_text())

# Filter list with date file names
for file in zip_files:
    if 'JC' in file:
        continue
    elif '2021' in file: 
        continue
    elif '2022' in file:
        continue
    else:            
        filter_files.append(file)    

# Sort the list
filter_files.sort()

The `webbrowser.open_new(url)` function will open url using the default browser.

In [ ]:
# Download New York City zip files
for file in filter_files: 
    webbrowser.open_new(url + file)
    sleep(5)

After downloading all of respective .zip files, we will unzip them and then relocate them from the default download folder to the project folder.  
<div class="alert alert-block alert-warning">
Here I suggest that you <b>change the dafult download location</b> (usually "C:\Users\your_name\Downloads") to the project folder from the Settings page in the browser.
</div>

In [8]:
down_loc = 'C:/Users/Hui/Downloads/'
project_loc = 'C:/Users/Hui/Desktop/Graduate_Project/CitiBike/data/'

#-----OPTION 1: Save in project folder-----
# Unzip files and clean up data folder
for file in filter_files:
    file_name = project_loc + file
    with ZipFile(file_name) as zip_ref:
        zip_ref.extractall(project_loc)
    os.remove(file_name)

    
#-----OPTION 2: Save in default folder-----
# Unzip files and clean up data folder
# for item in os.listdir(down_loc):
#     if item.endswith('.zip'):
#         file_name = down_loc + item
#         with ZipFile(file_name) as zip_ref:
#             zip_ref.extractall(down_loc)
#         os.remove(file_name)
        
# Move from Download folder to data folder
# for item in os.listdir(down_loc):
#     shutil.move(down_loc + item, project_loc)

In [21]:
# Reset the data type
files = os.listdir('C:/Users/Hui/Desktop/Graduate_Project/CitiBike/data/2013/')

for csv in files:
    df = pd.read_csv(f'C:/Users/Hui/Desktop/Graduate_Project/CitiBike/data/2013/{csv}')
    df = df.rename(columns=({'trip_duration':'tripduration',
                             'start_time':'starttime',
                             'stop_time':'stoptime',
                             'start_station_id':'start station id',
                             'start_station_name':'start station name',
                             'start_station_latitude':'start station latitude',
                             'start_station_longitude':'start station longitude',
                             'end_station_id':'end station id',
                             'end_station_name':'end station name',
                             'end_station_latitude':'end station latitude',
                             'end_station_longitude':'end station longitude',
                             'bike_id':'bikeid',
                             'user_type':'usertype',
                             'birth_year':'birth year',
                             'gender':'gender'}))
    df.to_csv(f'C:/Users/Hui/Desktop/Graduate_Project/CitiBike/data/2013/{csv}', index = None)

In [22]:
ny_files = sorted(glob('C:/Users/Hui/Desktop/Graduate_Project/CitiBike/data/2013/*.csv'))
ny_trip_data = pd.concat((pd.read_csv(file) for file in ny_files), ignore_index = True)
ny_trip_data.to_csv('C:/Users/Hui/Desktop/Graduate_Project/CitiBike/data/2013/ny_trip_data_2013.csv', index = False)

In [23]:
ny_trip_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6697019 entries, 0 to 6697018
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tripduration             int64  
 1   starttime                object 
 2   stoptime                 object 
 3   start station id         int64  
 4   start station name       object 
 5   start station latitude   float64
 6   start station longitude  float64
 7   end station id           int64  
 8   end station name         object 
 9   end station latitude     float64
 10  end station longitude    float64
 11  bikeid                   int64  
 12  usertype                 object 
 13  birth year               object 
 14  gender                   int64  
dtypes: float64(4), int64(5), object(6)
memory usage: 766.4+ MB


---
### 03. Building a PostgreSQL Database
Please establish a connection with PostgreSQL and create a new database.

In [24]:
# Connect to PostgreSQL
conn = psycopg2.connect("user=postgres password='password'");
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);

# Obtain in a DB Cursor
cursor = conn.cursor();
db_name = "citibike_data"

# Create DB in PostgreSQL
create_database = f"CREATE DATABASE {db_name};"
cursor.execute(create_database);
print("Database created successfully in PostgreSQL")

# Closing PostgreSQL connection
if(conn):
    cursor.close()
    conn.close()
    print("PostgreSQL connection is closed")

DuplicateDatabase: database "citibike_data" already exists


In [ ]:
# Building a table for New York City data
try:
    conn = psycopg2.connect(user = 'postgres',
                           password = 'password',
                           host = '127.0.0.1',
                           port = '5432',
                           database = 'citibike_data')
    
    cursor = conn.cursor()
    
    # Some data types have been amended below to account for all data in the csv (i.e. blank cells)
    create_table_query = '''CREATE TABLE new_york_city(
                            trip_duration INT,
                            start_time TIMESTAMP,
                            stop_time TIMESTAMP,
                            start_station_id INT,
                            start_station_name TEXT,
                            start_station_latitude FLOAT,
                            start_station_longitude FLOAT,
                            end_station_id INT,
                            end_station_name TEXT,
                            end_station_latitude FLOAT,
                            end_station_longitude FLOAT,
                            bike_id INT,
                            user_type TEXT,
                            birth_year TEXT,
                            gender INT
                            ); '''
    
    cursor.execute(create_table_query)
    conn.commit()
    print("Table created successfully in PostgreSQL")

except (Exception, psycopg2.DatabaseError) as error:
    print("Error while creating PostgreSQL table:", error)
finally:
    # closing database connection
    if (conn):
        cursor.close()
        conn.close()
        print("PostgreSQL connection is closed")

In [ ]:
# Populating the NYC table
try:
    conn = psycopg2.connect(user = 'postgres',
                           password = 'password',
                           host = '127.0.0.1',
                           port = '5432',
                           database = 'citibike_data')
    
    cursor = conn.cursor()
    
    with open('C:/Users/Hui/Desktop/Graduate_Project/CitiBike/data/2013/ny_trip_data_2013.csv', 'r') as data:
        next(data) # skip the header row
        cursor.copy_from(data, 'new_york_city', sep=',')
        
        conn.commit()
        
        print("Table updated successfully in PostgreSQL ")
        
except (Exception, psycopg2.DatabaseError) as error:
    print ("Error while updating PostgreSQL table:", error)
    
finally:
    if(conn):
            cursor.close()
            conn.close()
            print("PostgreSQL connection is closed")

Unsettled problem:  
- separate the data by years
- improve pd.read() function